### NLP Stop words 

- Importaciones

In [70]:
import pandas as pd
import re
import spacy
import spacy
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



- Obtención de los datos 

In [ ]:
# Cargar modelo spaCy para español
nlp = spacy.load("es_core_news_sm")
# Cargar el CSV
df = pd.read_csv("comentarios_clientes.csv", sep=",")
df.head()

,texto,etiqueta
0,El paquete llegó con el embalaje comprometido ...,neutral
1,Después de 3 semanas de espera el artículo nun...,negativo
2,La atención al cliente fue evasiva y poco reso...,negativo
3,Entrega express y producto en perfecto estado....,positivo
4,El sistema de seguimiento en línea mostró info...,negativo


- Limpieza y procesamiento de datos

In [73]:
def limpiar_texto(texto):
    # Convertir a minúsculas y eliminar puntuación
    texto = re.sub(r'[^\w\s]', '', texto.lower())

    # Procesar con spaCy
    doc = nlp(texto)

    # Filtrar stopwords y palabras cortas
    tokens = [token.lemma_ for token in doc if not token.is_stop and len(token.lemma_) > 2]

    return tokens




In [75]:
df["n_tokens"] = df["tokens"].apply(len)
df[["texto", "tokens", "n_tokens"]].head()

,texto,tokens,n_tokens
0,El paquete llegó con el embalaje comprometido ...,"[paquete, embalaje, comprometido, lograr, reem...",6
1,Después de 3 semanas de espera el artículo nun...,"[semana, esperar, artículo, despachar, pésimo,...",6
2,La atención al cliente fue evasiva y poco reso...,"[atención, cliente, evasivo, resolutivo, reclamo]",5
3,Entrega express y producto en perfecto estado....,"[entrega, express, producto, perfecto, superar...",6
4,El sistema de seguimiento en línea mostró info...,"[sistema, seguimiento, línea, mostrar, informa...",6


#### FASE 1: Personalización de la Lista

In [76]:
# Carga el modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Base de stopwords de spaCy
stopwords_es = nlp.Defaults.stop_words.copy()

# Quitar términos importantes
for palabra in ["no", "nunca", "tampoco", "pero", "aunque", "sin", "embargo"]:
    stopwords_es.discard(palabra)

# Añadir términos irrelevantes
palabras_a_añadir = ["producto", "cliente", "día", "hacer", "tener", "decir", "hola", "gracias", "pd"]
stopwords_es.update(palabras_a_añadir)



#### FASE 2: Análisis de frecuencia con Counter

In [79]:
df = pd.read_csv("./comentarios_clientes.csv")

# Unir todo el texto para frecuencia
all_text = " ".join(df["texto"].astype(str).str.lower())
tokens = [token.text for token in nlp(all_text) if token.is_alpha]
counter = Counter(tokens)

# Mostrar palabras que aparecen en más del 90% de las reseñas
umbral = len(df) * 0.9
comunes = [palabra for palabra, freq in counter.items() if freq >= umbral]
print("Términos repetidos en el 90%:", comunes)


Términos repetidos en el 90%: []


#### FASE 3: Prueba de función personalizada

In [80]:
def limpiar_texto(texto):
    doc = nlp(texto.lower())
    return [token.text for token in doc if token.is_alpha and token.text not in stopwords_es]

# Casos de prueba
texto1 = "No funciona bien, pero el diseño es bonito."
texto2 = "Nunca compré algo tan malo. Aunque el precio es bajo, no lo vale."

print("Texto 1:", limpiar_texto(texto1))
print("Texto 2:", limpiar_texto(texto2))




Texto 1: ['no', 'funciona', 'pero', 'diseño', 'bonito']
Texto 2: ['nunca', 'compré', 'malo', 'aunque', 'precio', 'no', 'vale']


#### Fase 4: Evaluación del Impacto

In [81]:

# Dataset de ejemplo
X = df["texto"]
y = df["etiqueta"]

# Sin limpieza
vectorizer1 = CountVectorizer(max_features=1000)
X_vect1 = vectorizer1.fit_transform(X)

# Con limpieza personalizada
X_clean = [" ".join(limpiar_texto(texto)) for texto in X]
vectorizer2 = CountVectorizer(max_features=1000)
X_vect2 = vectorizer2.fit_transform(X_clean)

# Entrenamiento
X1_train, X1_test, y1_train, y1_test = train_test_split(X_vect1, y, test_size=0.2, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_vect2, y, test_size=0.2, random_state=42)

clf1 = LogisticRegression()
clf1.fit(X1_train, y1_train)
acc1 = accuracy_score(y1_test, clf1.predict(X1_test))

clf2 = LogisticRegression()
clf2.fit(X2_train, y2_train)
acc2 = accuracy_score(y2_test, clf2.predict(X2_test))

print("Exactitud sin limpieza personalizada:", acc1)
print("Exactitud con limpieza personalizada:", acc2)



Exactitud sin limpieza personalizada: 0.6
Exactitud con limpieza personalizada: 0.4


- Muestra el resultado final del dataset

In [82]:
# Creamos una nueva columna con las reseñas limpias
df["texto_limpio"] = df["texto"].apply(lambda x: " ".join(limpiar_texto(x)))

# Mostramos el DataFrame final
pd.set_option('display.max_colwidth', None)  # Para que no corte el texto
display(df[["texto", "texto_limpio", "etiqueta"]])


,texto,texto_limpio,etiqueta
0,El paquete llegó con el embalaje comprometido aunque lograron reembolsarme rápidamente,paquete embalaje comprometido aunque lograron reembolsarme rápidamente,neutral
1,Después de 3 semanas de espera el artículo nunca fue despachado. Pésimo seguimiento,semanas espera artículo nunca despachado pésimo seguimiento,negativo
2,La atención al cliente fue evasiva y poco resolutiva en mi reclamo,atención evasiva resolutiva reclamo,negativo
3,Entrega express y producto en perfecto estado. Superó expectativas,entrega express perfecto superó expectativas,positivo
4,El sistema de seguimiento en línea mostró información inconsistente durante días,sistema seguimiento línea mostró información inconsistente,negativo
5,Materiales de calidad inferior a lo descrito en la ficha técnica,materiales calidad inferior descrito ficha técnica,negativo
6,El proceso de devolución resultó engorroso por requisitos excesivos,proceso devolución resultó engorroso requisitos excesivos,negativo
7,Artículo funcional pero con manual de instrucciones incompleto,artículo funcional pero manual instrucciones incompleto,neutral
8,Demoras recurrentes en la actualización del estado del envío,demoras recurrentes actualización envío,negativo
9,Embalaje ecológico y resistente que protegió bien el contenido,embalaje ecológico resistente protegió contenido,positivo
